In [1]:
import asyncio

from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.environment import Environment

from metagpt.const import MESSAGE_ROUTE_TO_ALL

2025-01-24 10:49:24.811 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to e:\wow-agent\notebook


In [2]:
classroom = Environment()

In [3]:
class Student(Role):

    name: str = "xiaoming"
    profile: str = "Student"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([WritePoem])
        self._watch([UserRequirement, ReviewPoem])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有记忆
        # logger.info(msg)
        poem_text = await WritePoem().run(msg)
        logger.info(f'student : {poem_text}')
        msg = Message(content=poem_text, role=self.profile,
                      cause_by=type(todo))

        return msg

class Teacher(Role):

    name: str = "laowang"
    profile: str = "Teacher"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([ReviewPoem])
        self._watch([WritePoem])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        todo = self.rc.todo

        msg = self.get_memories()  # 获取所有记忆
        poem_text = await ReviewPoem().run(msg)
        logger.info(f'teacher : {poem_text}')
        msg = Message(content=poem_text, role=self.profile,
                      cause_by=type(todo))

        return msg


In [4]:
class WritePoem(Action):

    name: str = "WritePoem"

    PROMPT_TEMPLATE: str = """
    Here is the historical conversation record : {msg} .
    Write a poem about the subject provided by human, Return only the content of the generated poem with NO other texts.
    If the teacher provides suggestions about the poem, revise the student's poem based on the suggestions and return.
    your poem:
    """

    async def run(self, msg: str):

        prompt = self.PROMPT_TEMPLATE.format(msg = msg)

        rsp = await self._aask(prompt)

        return rsp

class ReviewPoem(Action):

    name: str = "ReviewPoem"

    PROMPT_TEMPLATE: str = """

    Here is the historical conversation record : {msg} .
    Check student-created poems about the subject provided by human and give your suggestions for revisions. You prefer poems with elegant sentences and retro style.
    Return only your comments with NO other texts.
    your comments:
    """

    async def run(self, msg: str):

        prompt = self.PROMPT_TEMPLATE.format(msg = msg)

        rsp = await self._aask(prompt)

        return rsp

In [8]:
async def main(topic: str, n_round=3):

    classroom.add_roles([Student(), Teacher()])

    classroom.publish_message(
        Message(role="Human", content=topic, cause_by=UserRequirement,
                send_to='' or MESSAGE_ROUTE_TO_ALL),
        peekable=False,
    )

    while n_round > 0:
        # self._save()
        n_round -= 1 #如果n_round = 1 ，就只有学生写诗、然后老师没办法进行review
        logger.debug(f"max {n_round=} left.")

        await classroom.run()
    return classroom.history

await main(topic='写一副2025年农历蛇年春节对联')

2025-01-24 10:52:39.765 | INFO     | __main__:_act:12 - xiaoming(Student): ready to WritePoem


上联：金蛇狂舞迎春至  
下联：玉兔欢歌送岁归  

横批：蛇兔交替春常在

2025-01-24 10:52:40.587 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.
2025-01-24 10:52:40.588 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 0, completion_tokens: 0
2025-01-24 10:52:40.589 | INFO     | __main__:_act:18 - student : 上联：金蛇狂舞迎春至  
下联：玉兔欢歌送岁归  

横批：蛇兔交替春常在
2025-01-24 10:52:40.590 | INFO     | __main__:_act:35 - laowang(Teacher): ready to ReviewPoem



学生所作对联已颇具巧思，但可进一步追求典雅与古风。建议如下：

上联：金蛇婉转迎春至
下联：玉兔轻歌送岁归

横批：蛇兔相承春永驻

改动之处在于“婉转”与“轻歌”更添几分古雅之气，“相承”与“

2025-01-24 10:52:42.164 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.


...

2025-01-24 10:52:42.166 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 0, completion_tokens: 0
2025-01-24 10:52:42.166 | INFO     | __main__:_act:40 - teacher : 学生所作对联已颇具巧思，但可进一步追求典雅与古风。建议如下：

上联：金蛇婉转迎春至
下联：玉兔轻歌送岁归

横批：蛇兔相承春永驻

改动之处在于“婉转”与“轻歌”更添几分古雅之气，“相承”与“永驻”亦使横批更显庄重与深远。
2025-01-24 10:52:42.169 | INFO     | __main__:_act:12 - xiaoming(Student): ready to WritePoem


上联：金蛇婉转迎春至  
下联：玉兔轻歌送岁归  

横批：蛇兔相承春永

2025-01-24 10:52:42.978 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.
2025-01-24 10:52:42.978 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 0, completion_tokens: 0
2025-01-24 10:52:42.979 | INFO     | __main__:_act:18 - student : 上联：金蛇婉转迎春至  
下联：玉兔轻歌送岁归  

横批：蛇兔相承春永驻


驻


"\nHuman: wirte a poem about moon\nStudent: Beneath the cloak of night's deep shade,\nThe moon ascends, in silver made.\nA queen of shadows, softly cast,\nShe rules the night from her throne so vast.\n\nHer light, a whisper in the dark,\nGuides the wanderer with a gentle arc.\nA beacon to the lost and lone,\nShe shines, a guiding light, her own.\n\nIn her glow, the world is hushed,\nA tranquil scene, a peaceful plush.\nThe moon, a silent, watchful eye,\nThrough eons past, she never lies.\n\nFrom full to crescent, she transforms,\nA cycle ancient, never deforms.\nIn her phases, we find our fate,\nA constant in the changing state.\n\nSo here's to thee, oh moon so bright,\nA symbol of the endless night.\nThy light, a mystery, a dream,\nIn thy glow, all fears seem to seem.\nTeacher: The poem captures the moon's mystique well, yet a more retro style could enhance its elegance. Consider these revisions:\n\nBeneath the night's deep, shadowed veil,\nThe silver moon doth rise and sail.\nA shado